# 라이브러리 로드

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import gc
from tensorflow.keras.backend import clear_session

from matplotlib import rcParams, pyplot as plt

import numpy as np
import pandas as pd
import random as rn
import re

from pathlib import Path
import warnings

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence

from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import warnings 

In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

# 학습데이터 로드

In [4]:
seed = 2020
# np.random.seed(seed)
# rn.seed(seed)
# tf.random.set_seed(seed)
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
#                              inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)

In [5]:
data_dir = Path('../data/dacon-novel-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5

In [6]:
algorithm_name = 'lstm'
feature_name = 'feature'
model_name = f'{algorithm_name}_{feature_name}'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_Ver3_file = feature_dir / f'{feature_name}_Ver3.csv'
feature_Ver4_file = feature_dir / f'{feature_name}_Ver4.csv'


lstm_oof_pred_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
lstm_oof_pred_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
lstm_oof_pred_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
lstm_oof_pred_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'


lstm_test_pred_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'
lstm_test_pred_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'
lstm_test_pred_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'
lstm_test_pred_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'


lstm_submission_ver1_file = sub_dir / f'{model_name}_submission_Ver1.csv'
lstm_submission_ver2_file = sub_dir / f'{model_name}_submission_Ver2.csv'
lstm_submission_ver3_file = sub_dir / f'{model_name}_submission_Ver3.csv'
lstm_submission_ver4_file = sub_dir / f'{model_name}_submission_Ver4.csv'

## Ver 1 데이터 로드

In [7]:
dataset = pd.read_csv(feature_Ver1_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he was almost choking there was so much so muc...,3.0
1,your sister asked for it i suppose,2.0
2,she was engaged one day as she walked in peru...,1.0
3,the captain was in the porch keeping himself c...,4.0
4,have mercy gentlemen odin flung up his hands d...,3.0


In [8]:
# train set
Ver1_X = dataset.loc[dataset[target_col] != -1 , :]
Ver1_X.drop(columns=target_col,inplace=True,axis=1)
Ver1_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver1_y.astype(int)

# test set
Ver1_test = dataset.loc[dataset[target_col] == -1, :]
Ver1_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver1_X.shape, Ver1_y.shape, Ver1_test.shape)

(54879, 1) (54879,) (19617, 1)


## Ver 2 데이터 로드

In [9]:
dataset = pd.read_csv(feature_Ver2_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged one day walked perusing janes last let...,1.0
3,captain porch keeping carefully way treacherou...,4.0
4,mercy gentlemen odin flung hands dont write an...,3.0


In [10]:
# train set
Ver2_X = dataset.loc[dataset[target_col] != -1 , :]
Ver2_X.drop(columns=target_col,inplace=True,axis=1)
Ver2_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver2_y.astype(int)

# test set
Ver2_test = dataset.loc[dataset[target_col] == -1, :]
Ver2_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver2_X.shape, Ver2_y.shape, Ver2_test.shape)

(54879, 1) (54879,) (19617, 1)


## Ver3 데이터 로드

In [11]:
dataset = pd.read_csv(feature_Ver3_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged day walked perusing janes last letter ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,mercy gentlemen flung up hands dont write anyw...,3.0


In [12]:
# train set
Ver3_X = dataset.loc[dataset[target_col] != -1 , :]
Ver3_X.drop(columns=target_col,inplace=True,axis=1)
Ver3_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver3_y.astype(int)

# test set
Ver3_test = dataset.loc[dataset[target_col] == -1, :]
Ver3_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver3_X.shape, Ver3_y.shape, Ver3_test.shape)

(54879, 1) (54879,) (19617, 1)


## Ver4 데이터 로드

In [13]:
dataset = pd.read_csv(feature_Ver4_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he almost choking there so much so much he wan...,3.0
1,sister asked for it suppose,2.0
2,she engaged one day she walked perusing janes ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,have mercy gentlemen odin flung up hands dont ...,3.0


In [14]:
# train set
Ver4_X = dataset.loc[dataset[target_col] != -1 , :]
Ver4_X.drop(columns=target_col,inplace=True,axis=1)
Ver4_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver4_y.astype(int)

# test set
Ver4_test = dataset.loc[dataset[target_col] == -1, :]
Ver4_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver4_X.shape, Ver4_y.shape, Ver4_test.shape)

(54879, 1) (54879,) (19617, 1)


# 모델 학습

In [15]:
#파라미터 설정
vocab_size = 30000
embedding_dim = 128
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [16]:
def get_model():
    #가벼운 NLP모델 생성
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Dense(5, activation='softmax')
    ])
    
    # compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=.01))
    return model

In [17]:
datasets = [ (Ver1_X, Ver1_test, Ver1_y), (Ver2_X, Ver2_test, Ver2_y),
            (Ver3_X, Ver3_test, Ver3_y), (Ver4_X, Ver4_test, Ver3_y)]

lstm_oof_preds = []
lstm_test_preds = []

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

for number ,(X, test, y) in enumerate(datasets,1):
    print(f'start : {number}')
    # 토큰화
    X_train = np.array([x for x in X['text']])
    X_test = np.array([x for x in test['text']])
    y = np.array(y.values)
    
    tokenizer = Tokenizer(num_words = vocab_size)
    tokenizer.fit_on_texts(X_train)
    word_index = tokenizer.word_index
    
    # 시퀸스화 + 패딩
    train_sequences = tokenizer.texts_to_sequences(X_train)
    test_sequences = tokenizer.texts_to_sequences(X_test)
    trn = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
    tst = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)
    print(trn.shape, tst.shape)
    
    # oof , test 저장
    lstm_oof_pred = np.zeros((trn.shape[0], n_class))
    lstm_test_pred = np.zeros((tst.shape[0], n_class))
    
    for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
        print(f'traing model for CV #{i}')
        clf = get_model()
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=200,
            batch_size=1024,
            callbacks=[es])
        
        lstm_oof_pred[i_val, :] = clf.predict(trn[i_val])
        lstm_test_pred += clf.predict(tst) / n_fold
        
        del clf
        clear_session()
        gc.collect()
    
    lstm_oof_preds.append(lstm_oof_pred)
    lstm_test_preds.append(lstm_test_pred)
        
    print(f'end : {number}')

start : 1
(54879, 500) (19617, 500)
traing model for CV #1
Epoch 1/200
43/43 [==============================] - 23s 537ms/step - loss: 1.3609 - val_loss: 1.0124
Epoch 2/200
43/43 [==============================] - 22s 507ms/step - loss: 0.7708 - val_loss: 0.7674
Epoch 3/200
43/43 [==============================] - 22s 508ms/step - loss: 0.4995 - val_loss: 0.7320
Epoch 4/200
43/43 [==============================] - 22s 508ms/step - loss: 0.3544 - val_loss: 0.7609
Epoch 5/200
43/43 [==============================] - 22s 511ms/step - loss: 0.2760 - val_loss: 0.8571
Epoch 6/200
43/43 [==============================] - 22s 510ms/step - loss: 0.2265 - val_loss: 0.9220
Epoch 00006: early stopping
traing model for CV #2
Epoch 1/200
43/43 [==============================] - 23s 536ms/step - loss: 1.2759 - val_loss: 1.0086
Epoch 2/200
43/43 [==============================] - 22s 510ms/step - loss: 0.7133 - val_loss: 0.7749
Epoch 3/200
43/43 [==============================] - 22s 509ms/step - loss

In [18]:
for i,j in enumerate(lstm_oof_preds,1):
    print(f'ver{i} logloss = {log_loss(pd.get_dummies(y),j):8.4f}')
    print(f'ver{i} accuracy = {accuracy_score(y, np.argmax(j,axis=1))*100:8.4f}')

ver1 logloss =   0.7444
ver1 logloss =  73.6712
ver2 logloss =   0.7431
ver2 logloss =  72.7765
ver3 logloss =   0.7439
ver3 logloss =  72.5050
ver4 logloss =   0.7379
ver4 logloss =  73.3213


# 제출 파일 및 기타 파일 생성

In [19]:
# submission 파일 생성

sub = pd.read_csv(sample_file,index_col=0)

# Ver1 
sub[sub.columns] = lstm_test_preds[0]
sub.to_csv(lstm_submission_ver1_file)

# Ver2
sub[sub.columns] = lstm_test_preds[1]
sub.to_csv(lstm_submission_ver2_file)

# Ver3
sub[sub.columns] = lstm_test_preds[2]
sub.to_csv(lstm_submission_ver3_file)
           
# Ver4
sub[sub.columns] = lstm_test_preds[3]
sub.to_csv(lstm_submission_ver4_file)

In [20]:
# lstm_oof_pred 파일 생성

# Ver1
np.savetxt(lstm_oof_pred_ver1_file, lstm_oof_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(lstm_oof_pred_ver2_file, lstm_oof_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(lstm_oof_pred_ver3_file, lstm_oof_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(lstm_oof_pred_ver4_file, lstm_oof_preds[3],fmt='%.18f', delimiter=',')

In [21]:
# lstm_test_pred 파일 생성

# Ver1
np.savetxt(lstm_test_pred_ver1_file, lstm_test_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(lstm_test_pred_ver2_file, lstm_test_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(lstm_test_pred_ver3_file, lstm_test_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(lstm_test_pred_ver4_file, lstm_test_preds[3],fmt='%.18f', delimiter=',')